In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler

In [ ]:
df = pd.read_csv("/content/microchip.csv")

In [ ]:
def BoxMullerTransformation(r1, r2):
    z1 = np.sqrt(-2 * np.log(r1)) * np.cos(2 * np.pi * r2)
    z2 = np.sqrt(-2 * np.log(r1)) * np.sin(2 * np.pi * r2)
    return z1, z2

In [ ]:
def GDA(x, inv, mu, d):  
    res = 1 / ((2*np.pi)**(x.shape[0]/2) * d) * np.exp(-0.5 * np.dot(np.dot((x - mu), inv), (x - mu).T))
    return res

In [ ]:

X = df[["1", "2"]]
y = df[["class"]]
y = np.array(y)
X[['1', '2']] = MinMaxScaler().fit_transform(X[['1', '2']])
median = X['1'].median()
X['1'].replace(0, median)
median = X['2'].median()
X['2'].replace(0, median)

X = np.array(X)

r1, r2 = BoxMullerTransformation(X[:,0], X[:,1])

X_data = np.stack((r1, r2), axis=-1)

X_train, X_test, y_train, y_test = train_test_split(X_data,y, test_size = 0.3, random_state = 1)

y_mean = y.mean()
X_1 = []
X_0 = []

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:2969: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.loc._setitem_with_indexer((slice(None), indexer), value)
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:2935: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vie

In [ ]:
for i in range(X_train.shape[0]):
    if y_train[i] == 1:
        X_1.append(X_train[i])
    else:
        X_0.append(X_train[i])

In [ ]:
X_1 = np.vstack(X_1)
X_0 = np.vstack(X_0)


In [ ]:

def init(X_1, X_0):
    mean_1 = X_1.mean(axis = 0)
    mean_0 = X_0.mean(axis = 0)

    X_1 = X_1 - mean_1
    X_0 = X_0 - mean_0
    sig = (np.dot(X_1.T, X_1) + np.dot(X_0.T, X_0))/X.shape[0]
    s_inv = np.linalg.inv(sig)
    det = np.sqrt(np.linalg.det(sig))
    y_pred = []
    for i in range(X_test.shape[0]):
        prob_1 = GDA(X_test[i], s_inv, mean_1, det) * y_mean
        prob_0 = GDA(X_test[i], s_inv, mean_0, det) * (1 - y_mean)
    
        if prob_1 > prob_0:
            y_pred.append(1)
        else:
            y_pred.append(0)
        
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

In [ ]:
accuracy = init(X_1, X_0)
print(accuracy)

0.6944444444444444
